In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer

from safe_rlhf.datasets import PointwiseSafeDataset

model_name_or_path = "PKU-Alignment/alpaca-7b-reproduced"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

# Instead of "datasets=...", use the correct parameter:
train_dataset = PointwiseSafeDataset(
    [
        ("PKU-SafeRLHF/train", 0.01),
        # If you have multiple, you can specify [("Foo", 0.6), ("Bar", 0.4), ...]
    ],
    tokenizer=tokenizer,
    lazy_tokenization=False,
    seed=42,
)



[2025-03-24 19:28:15,210] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/chiche/miniconda3/envs/pda/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/chiche/miniconda3/envs/pda/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status
Preprocessing raw dataset...: 100%|██████████| 739/739 [00:00<00:00, 1494.16it/s]


In [2]:
train_dataset.data[213]["better_input_ids"].argmax()

tensor(57)

In [3]:
better_log_ratio =  torch.tensor(0.1)
worse_log_ratio =  torch.tensor(1.0)
torch.stack([torch.tensor([torch.exp(better_log_ratio),torch.exp(worse_log_ratio)]), torch.tensor([torch.exp(better_log_ratio),torch.exp(worse_log_ratio)])])

tensor([[1.1052, 2.7183],
        [1.1052, 2.7183]])

In [4]:
from torch.utils.data import DataLoader


train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=1,
    collate_fn=train_dataset.get_collator()
)

# 3. Access a batch
# The batch is a dictionary with Tensors for 'better_input_ids', 'worse_input_ids', etc.
batch = next(iter(train_dataloader))
print("Keys in batch:", batch.keys())

for key, val in batch.items():
    print(key, val.shape, val.dtype,val[0])



Keys in batch: dict_keys(['better_input_ids', 'better_attention_mask', 'worse_input_ids', 'worse_attention_mask', 'better_safe', 'worse_safe', 'index'])
better_input_ids torch.Size([1, 173]) torch.int64 tensor([    1, 22815, 29940,  4214,  8079,  8707,  5348, 29903,  8098, 29901,
         3148,  1001, 29901,  1128,   437,   306,  1207,   263, 18046,   280,
          310,  7123,   368, 27908,   304, 12088,   590,   289,  2209, 29973,
          319,  1799,  9047, 13566, 29901, 29924,   861,   292, 27908,   508,
          367,  5189, 29892,   577,   372,   338, 13622,   304,   671, 12128,
          772, 14125,   565,  3625, 29889,  4525,  4072,   310,   772, 14125,
          526, 12234,  5239,   408, 15382,   296,   470, 17564,  1634,   514,
         1237,   322,  5491,  1712,   263,   752,   618,  1316,   408,   380,
          719,  3049,   457, 29892,  5094,   273,   680,   470, 22004,  1601,
         2251,   680, 29889,   739,   338,  4100,   304, 16467,  1101,   278,
        11994,   

In [13]:
train_dataset.tokenizer

LlamaTokenizerFast(name_or_path='PKU-Alignment/alpaca-7b-reproduced', vocab_size=32000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	32000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [22]:
from safe_rlhf.models import AutoModelForScore, load_pretrained_models

reward_model, reward_tokenizer = load_pretrained_models(
    'PKU-Alignment/beaver-7b-v3.0-reward',
    model_max_length=512,
    auto_model_type=AutoModelForScore,
    padding_side='right',
    trust_remote_code=True,
    auto_model_kwargs={
        'score_type': 'reward',
        'do_normalize': True,
    },
)
reward_model.set_normalize(True)
reward_model.cuda()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacity of 47.53 GiB of which 14.06 MiB is free. Process 3723149 has 7.09 GiB memory in use. Including non-PyTorch memory, this process has 39.28 GiB memory in use. Process 3907803 has 1.11 GiB memory in use. Of the allocated memory 38.96 GiB is allocated by PyTorch, and 4.47 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [21]:
reward_model.model.embed_tokens.weight.size(0)


32001

In [ ]:
reward_model.model.embed_tokens.padding_idx


TypeError: 'int' object is not callable

In [7]:
reward_model.eval()
reward_model.requires_grad_(False)

LlamaForScore(
  (model): LlamaModel(
    (embed_tokens): Embedding(32001, 4096, padding_idx=32000)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-0

In [8]:
batch.keys()

dict_keys(['better_input_ids', 'better_attention_mask', 'worse_input_ids', 'worse_attention_mask', 'better_safe', 'worse_safe', 'index'])

In [9]:
reward = reward_model(
            batch["better_input_ids"].cuda(),
            attention_mask=batch["better_attention_mask"].cuda(),
        ).end_scores.squeeze(dim=-1)

In [10]:
from tqdm import tqdm
rewards = torch.zeros(
            (len(train_dataloader.dataset), train_dataloader.dataset.num_respones)
        ).cuda()
for batch in tqdm(train_dataloader):
    rewards[batch['index'], 0] = reward_model(
    batch["better_input_ids"].cuda(),
    attention_mask=batch["better_attention_mask"].cuda(),
    ).end_scores.squeeze(dim=-1)
    rewards[batch['index'], 1] = reward_model(
        batch["worse_input_ids"].cuda(),
        attention_mask=batch["worse_attention_mask"].cuda(),
    ).end_scores.squeeze(dim=-1)

  6%|▌         | 43/739 [00:03<01:00, 11.51it/s]


KeyboardInterrupt: 

In [18]:
rewards[batch['index']]

tensor([[-5.0835, -5.3851],
        [-0.4504,  1.7506]], device='cuda:0')

In [ ]:
batch

In [ ]:
cost_model, cost_tokenizer = load_pretrained_models(
    'PKU-Alignment/beaver-7b-v3.0-cost',
    model_max_length=512,
    auto_model_type=AutoModelForScore,
    padding_side='right',
    trust_remote_code=True,
    auto_model_kwargs={
        'score_type': 'cost',
        'do_normalize': True,
    },
)
cost_model.set_normalize(True)

cost_model.eval()


cost = cost_model(
        batch["better_input_ids"].cuda(),
        attention_mask=batch["better_attention_mask"].cuda(),
).end_scores.squeeze(dim=-1)